In [13]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta

In [15]:
def fetch_binance_ohlcv(symbol, interval, total_candles):
    url = 'https://api.binance.com/api/v3/klines' # Это URL Binance API для получения свечных данных (Klines / OHLCV).
    limit = 1000 # Binance позволяет максимум 1000 свечей за 1 запрос, это жёсткий лимит
    all_data = [] # Создаётся пустой список, куда мы будем складывать все полученные свечи.

    # Начинаем с "сейчас" и идём назад (Берётся текущее время в миллисекундах, чтобы начать загрузку с "сейчас" назад по времени)
    end_time = int(time.time() * 1000) #time.time() даёт секунды → умножаем на 1000, чтобы получить миллисекунды (API Binance этого требует).

    while len(all_data) < total_candles: # Главный цикл: продолжаем скачивать свечи, пока не набрали нужное количество
        params = {
            'symbol': symbol,
            'interval': interval,
            'limit': limit,
            'endTime': end_time # API вернёт свечи до этого момента
        }

        # Отправляем GET-запрос к Binance API и парсим ответ как JSON (список списков, каждая свеча — массив из 12 элементов)
        response = requests.get(url, params=params)
        data = response.json()

        # Если данные пустые или вернулся объект с полем 'code', значит ошибка (например, превышен лимит API) — тогда останавливаем цикл
        if not data or 'code' in data:
            print("Ошибка или лимит API:", data)
            break

        all_data = data + all_data  # prepend Добавляем новые свечи в начало списка (т.к. идём назад по времени, и хотим в итоге хронологически верную последовательность)
        #print(f"Получено {len(all_data)} свечей")

        # Новая конечная точка = время начала первой свечи
        first_open_time = data[0][0] # Берём время открытия самой ранней свечи из последней пачки
        end_time = first_open_time - 1 # Ставим его как endTime для следующего запроса — так мы загружаем ещё более старые данные в следующей итерации.

        time.sleep(0.2)  # чтобы не попасть под ограничение API

    # Обрезаем лишние данные, если скачалось чуть больше
    all_data = all_data[-total_candles:]

    # Преобразуем в DataFrame
    df = pd.DataFrame(all_data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base", "taker_buy_quote", "ignore"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms') # Преобразуем таймштамп из миллисекунд в datetime для удобства анализа
    return df[["timestamp", "open", "high", "low", "close", "volume"]]




In [16]:
%%time
# Скачиваем
df = fetch_binance_ohlcv(symbol="BTCUSDT", interval="1m", total_candles=500_000)

Получено 1000 свечей
Получено 2000 свечей
Получено 3000 свечей
Получено 4000 свечей
Получено 5000 свечей
Получено 6000 свечей
Получено 7000 свечей
Получено 8000 свечей
Получено 9000 свечей
Получено 10000 свечей
Получено 11000 свечей
Получено 12000 свечей
Получено 13000 свечей
Получено 14000 свечей
Получено 15000 свечей
Получено 16000 свечей
Получено 17000 свечей
Получено 18000 свечей
Получено 19000 свечей
Получено 20000 свечей
Получено 21000 свечей
Получено 22000 свечей
Получено 23000 свечей
Получено 24000 свечей
Получено 25000 свечей
Получено 26000 свечей
Получено 27000 свечей
Получено 28000 свечей
Получено 29000 свечей
Получено 30000 свечей
Получено 31000 свечей
Получено 32000 свечей
Получено 33000 свечей
Получено 34000 свечей
Получено 35000 свечей
Получено 36000 свечей
Получено 37000 свечей
Получено 38000 свечей
Получено 39000 свечей
Получено 40000 свечей
Получено 41000 свечей
Получено 42000 свечей
Получено 43000 свечей
Получено 44000 свечей
Получено 45000 свечей
Получено 46000 свеч

In [17]:
print(df.shape)
df

500000


,timestamp,open,high,low,close,volume
0,2024-05-24 09:14:00,67179.99000000,67186.00000000,67176.12000000,67176.12000000,5.12886000
1,2024-05-24 09:15:00,67176.13000000,67196.71000000,67176.12000000,67192.01000000,7.89970000
2,2024-05-24 09:16:00,67192.02000000,67192.02000000,67148.00000000,67148.00000000,9.62953000
3,2024-05-24 09:17:00,67148.00000000,67148.01000000,67126.47000000,67127.79000000,6.52463000
4,2024-05-24 09:18:00,67127.79000000,67184.01000000,67110.84000000,67184.01000000,26.64739000
...,...,...,...,...,...,...
499995,2025-05-06 14:29:00,94302.90000000,94317.72000000,94285.24000000,94317.72000000,18.13317000
499996,2025-05-06 14:30:00,94317.71000000,94344.80000000,94280.71000000,94280.72000000,15.67721000
499997,2025-05-06 14:31:00,94280.71000000,94294.48000000,94220.66000000,94241.53000000,11.25768000
499998,2025-05-06 14:32:00,94241.53000000,94294.85000000,94239.64000000,94255.64000000,17.83288000
